Required Installations:

In [1]:
# pip install crewai

In [1]:
import os
from crewai import Agent, Task, Crew
from langchain.llms import OpenAI
# from langchain.llms import Ollama

##### In order to use Crew.ai with Ollama, verify you have an ollama server running on your machine and set OpenAI's environment variables with these values:

##### export OPENAI_API_BASE='http://localhost:11434/v1'
##### export OPENAI_MODEL_NAME='mistral'
##### export OPENAI_API_KEY=NA

In [ ]:
# os.environ['OPENAI_API_KEY'] = '<YOUR_OPENAI_API_KEY>'


# Defining the agents
product_manager = Agent(
    role="Product Manager",
    goal="Oversee the project and ensure the game meets user needs and business goals",
    backstory="""
        You are an experienced Product Manager at the famous "Golan Heights Games Company" with a 
        knack for understanding user needs and translating them into product requirements.
        You have successfully launched several software products in the past and are known 
        for your strategic thinking and leadership skills.
    """,
    allow_delegation=False,
    verbose=True
)

software_engineer = Agent(
    role="Software Engineer",
    goal="Building quality software",
    backstory="""
        You are a talented Software Engineer at the famous "Golan Heights Games Company" with a passion 
        for coding and problem-solving.
        You enjoy working on challenging projects and have a strong background in game development.
        For the last 6 years you have been developing games using Python.
        You deliver fast and quality code and you are always open to a constructive criticism.
    """,
    allow_delegation=False,
    verbose=True
)

senior_software_engineer = Agent(
    role="Senior Software Engineer",
    goal="Lead the development team and ensure code quality and best practices",
    backstory="""
        You are a seasoned Senior Software Engineer at the famous "Golan Heights Games Company" with extensive 
        experience in software architecture and team leadership.
        You have a deep understanding of coding standards and best practices and are committed to mentoring junior developers.
        You are also a Python expert.
        You are commited to the company and to the quality of the product.
        You are kind but asertive and you will not hesitate to ask your workers to refactor
        their code when you believe its needed.
    """,
    allow_delegation=True,
    verbose=True
)

qa_engineer = Agent(
    role="QA Engineer",
    goal="Test the game to ensure it is free of bugs and provides a smooth user experience",
    backstory="""
        You are a meticulous QA Engineer at the famous "Golan Heights Games Company" with a sharp 
        eye for detail.
        You excel at finding and documenting bugs and have a strong commitment 
        to delivering high-quality software products.
    """,
    allow_delegation=True,
    verbose=True
)

# Defining the tasks
task1 = Task(
    description="Define the project scope and requirements for the Tic Tac Toe game.",
    expected_output="A detailed project scope document outlining game features, user stories, and acceptance criteria.",
    agent=product_manager
)

task2 = Task(
    description="Design and implement the game logic for Tic Tac Toe in Python using the PyGame package",
    expected_output="The full Well-documented code implementing the Tic Tac Toe game logic.",
    agent=software_engineer,
    dependencies=[task1]
)

task3 = Task(
    description="Review and optimize the game code to ensure high performance and maintainability.",
    expected_output="The full Optimized and well-structured game code with comments explaining key sections.",
    agent=senior_software_engineer,
    dependencies=[task2]
)

task4 = Task(
    description="""
        Test the game thoroughly to identify and document any bugs or issues that should be 
        fixed before the game is ready for deployment.
        Your Final answer must be the full python code, only the python code and nothing else.
    """,
    expected_output="The full final validated python code to run the game.",
    agent=qa_engineer,
    dependencies=[task3],
)

# Creating the crew
crew = Crew(
    agents=[product_manager, software_engineer, senior_software_engineer, qa_engineer],
    tasks=[task1, task2, task3, task4],
    verbose=2
)

# Kicking off the crew to start the project
result = crew.kickoff()

print("############")
print(result)